# Построение ML-продукта для оптимизации классификации заявок на оплату для сервиса Repetit.ru

**Описание проекта**
<br>Сервис Repetit.ru работает с большим количеством заявок от клиентов с данными о предмете, 
<br>желаемой стоимости, возрасте ученика, целью занятий и тд. 
<br>К сожалению, 7 из 8 не доходят до оплаты, при этом обработка заявки консультантом увеличивает конверсию в оплату на 30%. 
<br>Проблема в том, что консультантов не хватает на все заявки и получается, что чем больше заявок — 
<br>тем меньше конверсия из заявки в оплату и консультанты тратят время на бесперспективные заявки.

**Цель:**
- Сократить загрузку отдела консультантов на 30% 
- Увеличить конверсию в оплату на 15 п.п. через сегментацию заявок по вероятности их оплаты

**Задачи:**
<br>Разработать модель, которая по имеющейся информации о клиенте и заявке будет предсказывать вероятность оплаты заявки клиентом. 
<br>Заказчик хочет понять, какие заявки будут оплачены, а какие нет, чтобы одни обрабатывать вручную консультантами, а другие нет. 
<br>Оценка качества модели будет производиться с использованием `precision` и `ROC-AUC`.

**Ход исследования:**
- загрузка данных и ознакомление с ними,
- отбор подходящих признаков,
- EDA,
- создание новых признаков (при необходимости),
- отбор финального набора обучающих признаков,
- выбор и обучение моделей (разных архитектур),
- оценка качества предсказания лучшей модели на тестовой выборке,
- анализ важности признаков лучшей модели,
- создание сервиса в виде Docker Container,
- отчёт по проведённому исследованию.

**Описание данных**
- Заявки (orders*.csv)
  - order_date - дата создания 
  - subject_id - предмет
  - purpose - цель занятий
  - lesson_price - цена
  - lesson_duration - желаемая проодолжительность урока
  - home_metro_id - ближайшее метро
  - add_info - доп инфо
  - start_date
  - working_teacher_id
  - status_id - оплачена ли заявка (значения 6 и 13 говорят о факте оплаты заявки)
  - comments   
  - amount_to_pay
  - planned_lesson_number - клиент планирует N занятий
  - first_lesson_date - дата 1 занятия
  - coef - коэффициент
  - creator_id - кто создал заявку (id сотрудника или клиента)
  - pupil_category_new_id - возраст ученика
  - lessons_per_week - занятий а неделю
  - minimal_price
  - teacher_sex - пол репетитора
  - teacher_experience_from - опыт репетитора от
  - teacher_experience_to- опыт репетитора до
  - lesson_place_new - онлайн, у ученика, у учителя
  - pupil_knowledgelvl -уровень знаний ученика
  - teacher_age_from - желаемый возраст репеитора от
  - teacher_age_to - желаемый возраст репеитора от
  - chosen_teachers_only - не предлагать репетиторов кроме выбранных самостоятельно
  - no_teachers_available - на заявку нет подходящих репов
  - source_id - где создана заявка (какая часть сайта, не регион)
  - original_order_id - дублем какой заявки является эта заявка
  - client_id - айди клиента
  - additional_status_id
  - max_metro_distance - максимально готов ехать от метро
  - estimated_fee 
  - payment_date
  - test_group - аб тесты
  - is_display_to_teachers - хочет ли клиент получать отклики репетиторов

- Репетиторы (teacher_info.csv)
  - date_update
  - reg_date
  - birth_date
  - teaching_start_date
  - user_id - айди
  - is_email_confirmed
  - is_home_lessons
  - is_external_lessons
  - external_comments
  - lesson_duration - продолжит урока
  - lesson_cost - стоимость урока
  - status_id
  - status_relevant_date
  - status_school_id
  - status_college_id
  - status_display
  - russian_level_id
  - home_country_id
  - education
  - information
  - is_confirmed
  - is_display - показывается в каталоге
  - rating_id 
  - rating - рейтинг
  - comments
  - rules_confirmed_date
  - last_visited - послеждний визит
  - is_pupils_needed - открыт для заявок
  - is_cell_phone_confirmed
  - effective_rating - какой-то еще рейтинг
  - area_id
  - registrar_id
  - pupil_needed_date
  - sex
  - amount_to_pay - долг
  - is_remote_lessons
  - remote_comments
  - show_on_map
  - send_mailing
  - send_suitable_orders
  - rating_for_users - рейтинг 2
  - rating_for_admin - рейтинг 3
  - passport_id
  - is_edited
  - orders_allowed - разрешено назначать на заявки
  - display_days
  - verification_status_id
  - is_individual
  - partner_id
  - star_rating - рейтинг 4
  - rating_for_users_yesterday - рейтинг вчера
  - review_num - отзывы
  - relevance_date
  - is_display_at_partners
  - video_presentation_id - есть видеопрезентация
  - status_institution_id
  - Free_time_relevance_date


- Подходящие по фильтру репетиторы (suitable_teachers.csv)
  - tteacher_id - id репетитора
  - order_id - id заявки
  - contact_result    
  - enable_auto_assign - доступен ли репетитор к работе или заблокирован
    <br>(может ли репетитора назначить консультант и может ли он сам назначиться)
    <br>(значение известно на момент подачи заявки)
  - enable_assign - доступен ли репетитор к работе или заблокирован
    <br>(может ли репетитора назначить консультант и может ли он сам назначиться)
    <br>(значение известно на момент подачи заявки)


- Желаемые репетиторы (prefered_teachers_order_id.csv)
  <br>Репетиторы, которых выбрал клиент.
  - tteacher_id - id репетитора
  - order_id - id заявки

## Импорт библиотек

In [1]:
# import shap
# import phik
# import torch
import random
import warnings
# import numpy as np
import pandas as pd
# import lightgbm as lgb
import matplotlib.pyplot as plt

# from catboost import CatBoostClassifier, Pool
# from phik.report import plot_correlation_matrix
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import OrdinalEncoder, StandardScaler
# from sklearn.metrics import roc_curve, roc_auc_score, precision_score 

warnings.filterwarnings("ignore", category=UserWarning)

Подключение к GPU

In [3]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Установка констант

In [4]:
RANDOM_STATE = 42

Установка `random_state`

In [5]:
random.seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

## Загрузка данных

In [2]:
orders = pd.read_feather('data/orders.feather')
teachers_info = pd.read_feather('data/teachers_info.feather')
suitable_teachers = pd.read_feather('data/suitable_teachers.feather')
prefered_teachers_order_id = pd.read_feather('data/prefered_teachers_order_id.feather')

### Заявки

In [3]:
orders.info()
orders.sample()

<class 'pandas.core.frame.DataFrame'>
Index: 1191861 entries, 0 to 437216
Data columns (total 38 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   id                       1191861 non-null  int32  
 1   order_date               1191861 non-null  object 
 2   subject_id               1181529 non-null  float32
 3   purpose                  1025351 non-null  object 
 4   lesson_price             1191861 non-null  int32  
 5   lesson_duration          1191861 non-null  int32  
 6   lesson_place             31 non-null       object 
 7   home_metro_id            642447 non-null   float32
 8   add_info                 1044688 non-null  object 
 9   start_date               705578 non-null   object 
 10  working_teacher_id       705988 non-null   float32
 11  status_id                1191861 non-null  int32  
 12  comments                 341468 non-null   object 
 13  prefered_teacher_id      4 non-null        float

,id,order_date,subject_id,purpose,lesson_price,lesson_duration,lesson_place,home_metro_id,add_info,start_date,...,chosen_teachers_only,no_teachers_available,source_id,original_order_id,client_id,additional_status_id,max_metro_distance,estimated_fee,payment_date,is_display_to_teachers
380084,2223358,2022-09-28 10:58:38.000,21.0,Разделы: инженерная графика.\nКатегория ученик...,1000,60,None,205.0,Район: г. Долгопрудный.\nМесто проведения заня...,None,...,0,1,16,NaN,886298,8.0,NaN,None,None,1


In [8]:
print(f"количество полных дубликатов строк: {orders.duplicated().sum()}")
print(f"количество полных дубликатов в общем количестве строк: {orders.duplicated().sum() / orders.shape[0] * 100:.3f}%")

количество полных дубликатов строк: 0
количество полных дубликатов в общем количестве строк: 0.000%


Удаление обнаруженных повторов строк

In [7]:
orders = orders.drop_duplicates().reset_index(drop=True)

#### Выводы и наблюдения
- в таблице обнаружены дубликаты строк (0.076% от общего количества данных)
- принято решение об удалении найденных повторов ввиду их малочисленности

### Репетиторы

In [29]:
teachers_info.info()
teachers_info.sample(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307972 entries, 0 to 307971
Data columns (total 80 columns):
 #   Column                      Non-Null Count   Dtype   
---  ------                      --------------   -----   
 0   id                          307972 non-null  int32   
 1   date_update                 307972 non-null  category
 2   reg_date                    307972 non-null  category
 3   birth_date                  307972 non-null  category
 4   teaching_start_date         307972 non-null  category
 5   user_id                     307972 non-null  int32   
 6   is_email_confirmed          307972 non-null  int8    
 7   is_home_lessons             307972 non-null  int8    
 8   is_external_lessons         307972 non-null  int8    
 9   external_comments           90978 non-null   category
 10  lesson_duration             307972 non-null  int16   
 11  lesson_cost                 307972 non-null  int32   
 12  status_id                   307972 non-null  int8    
 13 

,id,date_update,reg_date,birth_date,teaching_start_date,user_id,is_email_confirmed,is_home_lessons,is_external_lessons,external_comments,...,verification_status_id,is_individual,partner_id,star_rating,rating_for_users_yesterday,review_num,relevance_date,is_display_at_partners,status_institution_id,free_time_relevance_date
296348,300981,2023-08-09 13:27:38.977,2023-08-07 12:59:11.427,1990-07-11 00:00:00.000,2019-09-01 00:00:00.000,1237080,0,0,0,NaN,...,0,NaN,NaN,4.300781,0.0,0,2023-08-07 12:59:11.427,0,NaN,NaN
43368,44785,2023-01-19 17:14:45.783,2012-10-23 15:40:27.000,1992-04-23 00:00:00.000,2008-09-01 00:00:00.000,45965,0,0,1,Ближайший район: м. Войковская. Выезд в районы...,...,1,0.0,NaN,4.300781,25.0,0,2017-01-01 00:00:00.000,1,NaN,NaN
141766,145161,2023-01-19 19:28:14.800,2018-07-23 10:28:46.863,1998-08-27 00:00:00.000,2016-09-01 00:00:00.000,371432,0,0,1,Ближайший район: м. Каховская.,...,0,NaN,ga_mow_own_sn_hv,4.300781,0.0,0,2018-07-23 10:28:46.863,0,NaN,NaN


In [31]:
print(f"количество полных дубликатов строк: {teachers_info.duplicated().sum()}")

количество полных дубликатов строк: 0


#### Выводы и наблюдения
- в таблице полных дубликатов строк не обнаружено

### Подходящие по фильтру репетиторы

In [5]:
suitable_teachers.info()
suitable_teachers.sample(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20271186 entries, 0 to 20271185
Data columns (total 5 columns):
 #   Column              Dtype   
---  ------              -----   
 0   teacher_id          int32   
 1   order_id            int32   
 2   contact_result      category
 3   enable_auto_assign  int8    
 4   enable_assign       int8    
dtypes: category(1), int32(2), int8(2)
memory usage: 232.3 MB


,teacher_id,order_id,contact_result,enable_auto_assign,enable_assign
8828660,147450,2525666,.,0,0
16177678,237441,2813560,NaN,0,0
9724241,19499,2557037,.,0,0


In [14]:
print(f"количество полных дубликатов строк: {suitable_teachers.duplicated().sum()}")
print(f"""количество полных дубликатов в общем количестве строк: {suitable_teachers.duplicated().sum() / 
suitable_teachers.shape[0] * 100:.4f}%""")

количество полных дубликатов строк: 0
количество полных дубликатов в общем количестве строк: 0.0000%


Удаление обнаруженных повторов

In [13]:
suitable_teachers = suitable_teachers.drop_duplicates().reset_index(drop=True)

In [25]:
print(f"уникальные записи колонки contact_result: {suitable_teachers['contact_result'].unique()}")
print(f"количество уникальных записей колонки contact_result: {suitable_teachers['contact_result'].nunique()}")

уникальные записи колонки contact_result: ['Репетитор согласился', 'Репетитор положил трубку, либо обрыв связи (O..., ',', NaN, 'Не дозвонились (Oktell)', ..., 'Богородский', '23 екат', 'Репетитор согласился с дополнительными услови..., 'Репетитор согласился с дополнительными услови..., 'к ученику?']
Length: 10867
Categories (10866, object): ['\nРепетитор оставил запрос : ', '\nРепетитор оставил запрос : \nРепетитор оста..., '\nРепетитор оставил запрос : Договорились о з..., '\nРепетитор оставил запрос : О занятии догово..., ..., '№', '№1 анкета понравилась, кл пока не хочет конта..., '№2 анкета не понравилась', '№3 анкета отправлена,  кл еще не смотрела']
количество уникальных записей колонки contact_result: 10866


In [23]:
print(f"уникальные записи колонки enable_auto_assign: {suitable_teachers['enable_auto_assign'].unique()}")
print(f"уникальные записи колонки enable_assign: {suitable_teachers['enable_assign'].unique()}")

уникальные записи колонки enable_auto_assign: [1 0]
уникальные записи колонки enable_assign: [1 0]


#### Выводы и наблюдения
- в таблице обнаружены дубликаты строк (0.0002% от общего количества данных)
- принято решение об удалении найденных повторов ввиду их малочисленности
- к категориальным признакам относятся: enable_auto_assign и enable_assign
- в признаке contact_result внесена информация в произвольном формате,
  <br>имеются совсем неинформативные записи ('23 екат', '...')

### Желаемые репетиторы (которых выбрал клиент)

In [9]:
prefered_teachers_order_id.info()
prefered_teachers_order_id.sample(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1135198 entries, 0 to 1135197
Data columns (total 2 columns):
 #   Column      Non-Null Count    Dtype
---  ------      --------------    -----
 0   order_id    1135198 non-null  int32
 1   teacher_id  1135198 non-null  int32
dtypes: int32(2)
memory usage: 8.7 MB


,order_id,teacher_id
198577,1760293,38193
219191,2191747,81008
1089470,2388892,203361


In [21]:
print(f"количество полных дубликатов строк: {prefered_teachers_order_id.duplicated().sum()}")
print(f"количество полных дубликатов в общем количестве строк: {prefered_teachers_order_id.duplicated().sum() / prefered_teachers_order_id.shape[0] * 100:.3f}%")

количество полных дубликатов строк: 16
количество полных дубликатов в общем количестве строк: 0.001%


Удаление обнаруженных повторов

In [24]:
prefered_teachers_order_id = prefered_teachers_order_id.drop_duplicates().reset_index(drop=True)

#### Выводы и наблюдения
- в таблице обнаружены дубликаты строк (0.001% от общего количества данных)
- принято решение об удалении найденных повторов ввиду их малочисленности

## Выбор лучшей модели

In [ ]:
dict_sum = {'Baseline': [roc_auc_baseline, precision_baseline],
         'Catboost': [roc_auc_cb, precision_cb],
         'LightGBM': [roc_auc_lgb, precision_lgb]}
summary_data = (pd.DataFrame.from_dict(dict_sum, orient='index',
                                      columns=["roc-auc", "precision"]).reset_index()
                                      .rename(columns={'index': 'model'})
                                      )
summary_data

## Создание файла зависимостей

In [ ]:
# !pipreqsnb operator_performance_analytics.ipynb

## Заключение